In [75]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import binom, norm
from statsmodels.stats.proportion import proportion_confint, proportions_ztest, samplesize_confint_proportion

# Confidence Intervals for Proportions

**Correct Interpretation**

Ex: 95% confidence intervval estimate of population proportion being (0.5, 0.6).

Interpreation: If we select many random samples, of the same sample size, and construct confidence intervals for them all.
95% of the confidence intervals will contain the population proportion.
If we repeat this process over and over, we should end up creating many confidence intervals and a "confidence level" proportion of them will contain the true population proportion.

Bad interpreation: There is a 95% chance the true population proportion will fall in the range of the confidence interval.

**Margin of Error**

Note: this is for Wald confidence intervals.

When a single random sample is used to estimate a population proportion, you can calculate the maximum likely amount of the error as such

$$
E = z_{\alpha / 2} \sqrt{ \frac{\hat p \hat q}{n} }
$$

Here $z_{\alpha / 2}$ is the critical value for the given confidence level $\alpha$.
The margin of error is the probability of $1 - \alpha$ (i.e., 95%) that the difference between the sample proportion and the population proportion is equal to or less than the margin of error.

This formula is one of the ways to build a confidence interval.

## Coverage Probability

The coverage probability of a confidence interval is the proportion in which our confidence intevals actually contain the population proportion.
The margin of error expression we offer above is for a Wald confidence interval whose coverage probability is equal to or less than the desired confidence interval.

* Plus Four Method is better than Wald and it just adds 2 successes and 2 failures to the sample size.
* Wilson Score is also better than Wald but the margin of error formula is more complicated (and more complex to invert to estimate sample sizes).
* Clopper-Pearson Method is exact because it makes use of a binomial distribution instead of using the normal distribution as an approximation. **Stats Model does support this method, set "method=beta"**.

## Approximating a Binomial Distribution with a Normal One

$$
\mu = np
$$

$$
\sigma = \sqrt{npq}
$$

Make sure that $np \geq 5$ and $nq \geq 5$ (or greater than 10).

## References

* Statsmodel index https://www.statsmodels.org/dev/stats.html
* Confidence interval for binomial proportions https://www.statsmodels.org/dev/generated/statsmodels.stats.proportion.proportion_confint.html
* Find sample size to get desired confidence interval length https://www.statsmodels.org/dev/generated/statsmodels.stats.proportion.samplesize_confint_proportion.html

In [84]:
def proportion_margin_of_error(sample_proportion, sample_size, confidence_lvl=0.95) -> float:
    """ Wald confidence interval
    """
    alpha = 1 - confidence_lvl
    critical_value = norm.ppf(confidence_lvl + (alpha / 2.0)) # We want the Z score from the desired covered area.
    
    q = 1.0 - sample_proportion
    return critical_value * np.sqrt(sample_proportion * q / sample_size)

def binom_estimated_mu(p, n) -> float:
    return n*p

def binom_estimated_sigma(p, n) -> float:
    return np.sqrt(n * p * (1.0-p))

In [45]:
# A gallup poll showed 43% of adults used facebook. Sample size was 1_487.

margin_err = proportion_margin_of_error(0.43, 1_487, confidence_lvl=0.95)

ci_low, ci_high = proportion_confint(
    count=0.43 * 1_487,  # Number of "successes"
    nobs=1_487,  # Number of trials.
    alpha=(1 - 0.95))

print(f"margin of error = {round(margin_err, 3)}")
print(f"CI low and CI high with estimated margin of error: ({round(0.43 - margin_err, 3)}, {round(0.43 + margin_err, 3)})")
print(f"CI low and CI high from statsmodel: ({round(ci_low, 3)}, {round(ci_high, 3)})")

margin of error = 0.025
CI low and CI high with estimated margin of error: (0.405, 0.455)
CI low and CI high from statsmodel: (0.405, 0.455)


In [73]:
# This works because here we can use a normal distribution to approximate the binomial distribution.
norm.interval(0.95, loc=binom_estimated_mu(0.43, 1_487), scale=binom_estimated_sigma(0.43, 1_487))

(601.9924600596482, 676.8275399403517)

In [48]:
binom.interval(0.95, n=1_487, p=0.43)

(602.0, 677.0)

In [74]:
# Both of the above give the number of successes, not the proportion.
print(f"({round(602/1_487, 3)}, {round(677/1_487, 3)})")

(0.405, 0.455)


In [86]:
# Out of 71 people, 70% were abstinent from smoking after 8 weeks of nicotine patches.
# 95% confidence interval 58% to 81%.

ci_low, ci_high = proportion_confint(
    count=50, #0.7 * 71 - Number of "successes"
    nobs=71,  # Number of trials.
    alpha=(1 - 0.95))

ci_low_exact, ci_high_exact = proportion_confint(
    count=50, #0.7 * 71 - Number of "successes"
    nobs=71,  # Number of trials.
    method="beta",
    alpha=(1 - 0.95))

estimate_margin_err = (ci_high - ci_low) / 2.0
estimate_population_p = (ci_high + ci_low) / 2.0

estimate_margin_err_exact = (ci_high_exact - ci_low_exact) / 2.0
estimate_population_p_exact = (ci_high_exact + ci_low_exact) / 2.0

print(f"CI low and CI high from statsmodel: ({round(ci_low, 3)}, {round(ci_high, 3)})")
print(f"margin of error: {round(estimate_margin_err, 3)}")
print(f"estimate of population proportion: {round(estimate_population_p, 3)}")

print(f"CI low and CI high from statsmodel using Clopper-Pearson: ({round(ci_low_exact, 3)}, {round(ci_high_exact, 3)})")
print(f"margin of error using Clopper Pearson: {round(estimate_margin_err_exact, 3)}")
print(f"estimate of population proportion using Clopper-Pearson: {round(estimate_population_p_exact, 3)}")

CI low and CI high from statsmodel: (0.598, 0.81)
margin of error: 0.106
estimate of population proportion: 0.704
CI low and CI high from statsmodel using Clopper-Pearson: (0.584, 0.807)
margin of error using Clopper Pearson: 0.111
estimate of population proportion using Clopper-Pearson: 0.695


## Determining Sample Size

In [85]:
def estimate_ci_sample_size(margin_of_error, proportion=0.5, confidence_lvl=0.95) -> int:
    """ Based on Wald confidence interval
    """
    alpha = 1 - confidence_lvl
    critical_value = norm.ppf(confidence_lvl + (alpha / 2.0)) # We want the Z score from the desired covered area.
    
    q = 1 - proportion
    return critical_value**2 * proportion * q / (margin_of_error**2)

In [83]:
# How many people to build a CI with 95% confidence and have an error of no more than 3 percentage points.
estimate_sample_size = estimate_ci_sample_size(
    margin_of_error=0.03,
    proportion=0.5,  # 0.5 maximizes the value.
    confidence_lvl=0.95)

estimate_n = samplesize_confint_proportion(proportion=0.5, half_length=0.03, alpha=(1-0.95))

print(f"estimated sample size based on margin of error: {estimate_sample_size}")
print(f"estimate sample size from statsmodel: {estimate_n}")

estimated sample size based on margin of error: 1067.0718946372572
estimate sample size from statsmodel: 1067.0718946372572


# Confidence Intervals for Means